In [1]:
import sys
sys.path.append('../')
import lemur

In [2]:
a = lemur.tensor([i for i in range(1,9)], (1,1,2,2,2),  requires_grad=True)
b = lemur.tensor([i for i in range(1+10,9+10)], (1,1,2,2,2),  requires_grad=True)
v = lemur.tensor([0,1,2,4,3])
a_p = a.permute(v)
c = b * a_p
dm = lemur.tensor([0,0,0,0,0])
d = c.sum(dm)

In [3]:
d.backward()

In [4]:
a

tensor: 0x143e915e0
  requires_grad = true
  comes_from:
  [NULL expression]
  k:
  kernel_tensor: 0x143e3fd00
    length     = 8
    shape      = [1, 1, 2, 2, 2]
    stride     = [8, 8, 4, 2, 1]
    computed   = false
    data:
      tensor([[[[[1.0000, 2.0000],[3.0000, 4.0000]],[[5.0000, 6.0000],[7.0000, 8.0000]]]]])
  grad:
  kernel_tensor: 0x143e82640
    length     = 8
    shape      = [1, 1, 2, 2, 2]
    stride     = [8, 8, 4, 2, 1]
    computed   = false
    data:
      tensor([[[[[11.0000, 13.0000],[12.0000, 14.0000]],[[15.0000, 17.0000],[16.0000, 18.0000]]]]])

In [6]:
import torch
at = torch.tensor([[[[[1.0000, 2.0000],[3.0000, 4.0000]],[[5.0000, 6.0000],[7.0000, 8.0000]]]]],  requires_grad=True, dtype=float)
bt = torch.tensor([[[[[11.0000, 12.0000],[13.0000, 14.0000]],[[15.0000, 16.0000],[17.0000, 18.0000]]]]],  requires_grad=True, dtype=float)
at_p = at.permute([0,1,2,4,3])
ct = bt * at_p
dt = ct.sum()
dt.backward()
at.grad, bt.grad

(tensor([[[[[11., 13.],
            [12., 14.]],
 
           [[15., 17.],
            [16., 18.]]]]], dtype=torch.float64),
 tensor([[[[[1., 3.],
            [2., 4.]],
 
           [[5., 7.],
            [6., 8.]]]]], dtype=torch.float64))

In [8]:
torch.empty((1,2,3,4,5))

tensor([[[[[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],

          [[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],

          [[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]]],


         [[[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],

          [[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]],

          [[0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0.]]]]])

In [3]:
x = lemur.tensor([i for i in range(1,33)], (2,2,2,2,2),  requires_grad=True)
y = x.sum(lemur.tensor([0,0,0,0,0]))
y

tensor: 0x154797690
  requires_grad = true
  k:
  kernel_tensor: 0x134740030
    length     = 1
    shape      = [1, 1, 1, 1, 1]
    stride     = [1, 1, 1, 1, 1]
    computed   = false
    data:
      tensor([[[[[528.0000]]]]])
  grad:
  [NULL kernel_tensor]

In [ ]:
y.backward()

: 

In [ ]:
x = lemur.tensor([1.0], requires_grad=True)
y = lemur.tensor([4.0], requires_grad=True)

z = x + y  
w = z * x  
w = w.relu() 

w.backward() 
print("Result from main:", w)

Result from main: tensor: 0x156a0f1b0
  requires_grad = true
  k:
  kernel_tensor: 0x156aa0f80
    length     = 1
    shape      = [1, 1, 1, 1, 1]
    stride     = [1, 1, 1, 1, 1]
    computed   = false
    data:
      tensor([[[[[5.0000]]]]])
  grad:
  [NULL kernel_tensor]



In [36]:
shape = [1,1,1,1,10]

In [37]:
%%timeit
x = lemur.LemurTensor(shape=shape, requires_grad=False)
y = lemur.LemurTensor(shape=shape, requires_grad=False)
x + y  

5.43 µs ± 781 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [3]:
import torch 

In [39]:
%%timeit
x = torch.empty(shape)
y = torch.empty(shape)
x + y  

2.81 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [11]:
t0 = torch.tensor([10.], requires_grad=True)
t1 = torch.tensor([12.], requires_grad=True)
t2 = torch.tensor([-15.], requires_grad=True)
t4 = t0 * t1 + t2
t4


tensor([105.], grad_fn=<AddBackward0>)

In [12]:
t4.backward()

In [13]:
t0.grad, t1.grad, t2.grad

(tensor([12.]), tensor([10.]), tensor([1.]))

In [25]:
a = lemur.tensor([[10.,12.],[-1.,-4.]], requires_grad = True)
b = lemur.rand_like(a)
c = (a @ b).mean()
c.backward()
a.grad

tensor([[0.4100, 0.2642],
        [0.4100, 0.2642]])

In [26]:
c

tensor(5.8041, grad_fn=<MeanBackward0>)

In [ ]:
a = lemur.tensor([...], requires_grad = True)
b = lemur.tensor([...], requires_grad = True)
c = (a + b).relu()

In [31]:
x = torch.tensor([[1], [2], [3]])
x.size(), x.stride()

(torch.Size([3, 1]), (1, 1))

In [35]:
x.expand(3, 4)


tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]])

In [47]:
x.retain_grad()
x.grad

In [62]:
x = torch.tensor([[1],[2]]).expand((2,10))
x.shape, x.stride()

(torch.Size([2, 10]), (1, 0))

In [64]:
y = x.reshape(20)

In [67]:
y.shape, y.stride

(torch.Size([20]), <function Tensor.stride>)

In [86]:
x = torch.tensor([[10.]]).expand(1,10)
x.requires_grad = True
x.stride()

(1, 0)

In [88]:
x.sum().backward()
x.grad.stride()

(10, 1)

In [99]:
x = torch.rand((3,4)).t() #(1,4)
x.requires_grad = True


In [100]:
x.sum().backward()

In [101]:
x.grad.stride()

(1, 4)

In [83]:
float arr[27]
shape (3,3,3) 
stride (9,3,1)

In [ ]:
t0 -> 1,1 
   -> length = 1

t1 -> 1,10 
   -> length = 1   
   -> comesfrom -> t0, expand, 1 10






In [73]:
x.stride()

(1, 0)

In [3]:
import torch

In [152]:
x = torch.randn((3,1), requires_grad=True)
x2 = torch.randn((3,1), requires_grad=True)

x.stride()

(1, 1)

In [153]:
y = (x*x2).expand(3,4)
y.stride()

(1, 0)

In [154]:
y.sum().backward()

In [159]:
x.grad, x2*4

(tensor([[-1.0801],
         [-0.6021],
         [-2.2774]]),
 tensor([[-1.0801],
         [-0.6021],
         [-2.2774]], grad_fn=<MulBackward0>))

In [236]:
torch.empty((1,2,3,4,5)).permute(1,2,3,4,0).stride()


(60, 20, 5, 1, 120)

In [176]:
x = torch.tensor([[1]]).expand(1,10)
x.stride()


(1, 0)

In [177]:
y = x.relu()
y.stride()

(10, 1)

In [168]:
x

tensor([[1., 0.]])

In [234]:
x = torch.tensor([[0.,0.]]).permute(1,0)
x.requires_grad = True
x, x.stride(), x.is_contiguous()


(tensor([[0.],
         [0.]], requires_grad=True),
 (1, 2),
 True)

In [217]:
x.sum().backward()

In [218]:
x.grad.stride(), x.stride()

((1, 2), (1, 2))

In [222]:
x.is_contiguous()

True

In [220]:
x.contiguous().stride()

(1, 2)

In [241]:
a = torch.arange(8).view(1,4,2)
a.stride()

(8, 2, 1)

In [233]:
a.permute(1,0).is_contiguous()

False

In [52]:
a = torch.arange(1,33, dtype=float, requires_grad=True)
a.view(2,2,2,2,2).sum().backward()
a.grad

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)

In [3]:
x = lemur.tensor([i for i in range(1,33)], (2,2,2,2,2),  requires_grad=True)
y = x.sum(lemur.tensor([0,0,0,0,0]))
y


tensor: 0x126e1d020
  requires_grad = true
  k:
  kernel_tensor: 0x146fb6510
    length     = 1
    shape      = [1, 1, 1, 1, 1]
    stride     = [1, 1, 1, 1, 1]
    computed   = false
    data:
      tensor([[[[[528.0000]]]]])
  grad:
  [NULL kernel_tensor]

In [ ]:
y.backward()

In [8]:
x

tensor: 0x1200056c0
  requires_grad = true
  k:
  kernel_tensor: 0x1200062e0
    length     = 32
    shape      = [2, 2, 2, 2, 2]
    stride     = [16, 8, 4, 2, 1]
    computed   = false
    data:
      tensor([[[[[1.0000, 2.0000],[3.0000, 4.0000]],[[5.0000, 6.0000],[7.0000, 8.0000]]],[[[9.0000, 10.0000],[11.0000, 12.0000]],[[13.0000, 14.0000],[15.0000, 16.0000]]]],[[[[17.0000, 18.0000],[19.0000, 20.0000]],[[21.0000, 22.0000],[23.0000, 24.0000]]],[[[25.0000, 26.0000],[27.0000, 28.0000]],[[29.0000, 30.0000],[31.0000, 32.0000]]]]])
  grad:
  kernel_tensor: 0x120006270
    length     = 32
    shape      = [2, 2, 2, 2, 2]
    stride     = [16, 8, 4, 2, 1]
    computed   = false
    data:
      tensor([[[[[1.0000, 1.0000],[1.0000, 1.0000]],[[1.0000, 1.0000],[1.0000, 1.0000]]],[[[0.0000, 0.0000],[0.0000, 0.0000]],[[0.0000, 0.0000],[0.0000, 0.0000]]]],[[[[0.0000, 0.0000],[0.0000, 0.0000]],[[0.0000, 0.0000],[0.0000, 0.0000]]],[[[0.0000, 0.0000],[0.0000, 0.0000]],[[0.0000, 0.0000],[0.0000, 0.00

In [246]:
a.sum(0,keepdim=True).sum(1,keepdim=True).sum(2,keepdim=True).sum(3,keepdim=True)


tensor([[[[[256, 272]]]]])

In [256]:
a.sum(0,keepdim=True).sum(1,keepdim=True).sum(3,keepdim=True).sum(4,keepdim=True)

tensor([[[[[232.]],

          [[296.]]]]])

In [ ]:
(2,2,2,2,2) -> (1,2,2,2,1)

In [43]:
s = (5,5,50,5,5)
a_t = torch.empty(s)
a_l = lemur.LemurTensor(shape=s, requires_grad=False)



In [44]:
%%timeit
b = a_t.sum(0,keepdim=True).sum(1,keepdim=True).sum(2,keepdim=True).sum(3,keepdim=True)

14 µs ± 196 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
%%timeit
b = a_l.sum(lemur.tensor([0,0,0,0,0])) 

54.2 µs ± 211 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [2]:
import torch
torch.tensor([1,2]).expand(10)

RuntimeError: The expanded size of the tensor (10) must match the existing size (2) at non-singleton dimension 0.  Target sizes: [10].  Tensor sizes: [2]